In [7]:
import numpy as np 
import pandas as pd
import networkx as nx #create and store graph
from node2vec import Node2Vec #To run node2vec algorithm
import os

In [8]:
df=pd.read_csv('netflix_titles.csv')

In [9]:
df=df.dropna()
df=df.drop(['description'],axis=1)
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies"
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies
6,70304989,Movie,Automata,Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada","September 8, 2017",2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers"
7,80164077,Movie,Fabrizio Copano: Solo pienso en mi,"Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,"September 8, 2017",2017,TV-MA,60 min,Stand-Up Comedy
9,70304990,Movie,Good People,Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden","September 8, 2017",2014,R,90 min,"Action & Adventure, Thrillers"


In [10]:
df['title'].is_unique

False

In [11]:
df2=df.groupby(['title']).count()
print(df2[df2['show_id']>1][0:2]) 
#Print the lists of titles appearing more than one time

             show_id  type  director  cast  country  date_added  release_year  \
title                                                                           
Benji              2     2         2     2        2           2             2   
Blood Money        2     2         2     2        2           2             2   

             rating  duration  listed_in  
title                                     
Benji             2         2          2  
Blood Money       2         2          2  


In [12]:
df[df['title']=='Benji']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
1969,296682,Movie,Benji,Joe Camp,"Benji, Deborah Walley, Peter Breck, Edgar Buch...",United States,"March 6, 2018",1974,G,86 min,"Children & Family Movies, Classic Movies"
2159,80204923,Movie,Benji,Brandon Camp,"Gabriel Bateman, Darby Camp, Kiele Sanchez, Je...","United Arab Emirates, United States","March 16, 2018",2018,TV-PG,87 min,"Children & Family Movies, Dramas"


In [13]:
df['title']=df['title']+', '+df['date_added']
df['title'].is_unique

False

In [14]:
df[df['title']=='The Silence, March 1, 2018']

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
2387,80244078,Movie,"The Silence, March 1, 2018",Gajendra Ahire,"Raghuvir Yadav, Nagraj Manjule, Anjali Patil, ...",India,"March 1, 2018",2017,TV-MA,90 min,"Dramas, International Movies"
2388,80238292,Movie,"The Silence, March 1, 2018",Gajendra Ahire,"Raghuvir Yadav, Nagraj Manjule, Anjali Patil, ...",India,"March 1, 2018",2015,TV-MA,91 min,"Dramas, International Movies"


In [15]:
df.drop_duplicates(subset=['title'],keep = False, inplace = True) 
df['title'].is_unique

True

In [16]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in
0,81145628,Movie,"Norm of the North: King Sized Adventure, Septe...","Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies"
4,80125979,Movie,"#realityhigh, September 8, 2017",Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies
6,70304989,Movie,"Automata, September 8, 2017",Gabe Ibáñez,"Antonio Banderas, Dylan McDermott, Melanie Gri...","Bulgaria, United States, Spain, Canada","September 8, 2017",2014,R,110 min,"International Movies, Sci-Fi & Fantasy, Thrillers"
7,80164077,Movie,"Fabrizio Copano: Solo pienso en mi, September ...","Rodrigo Toro, Francisco Schultz",Fabrizio Copano,Chile,"September 8, 2017",2017,TV-MA,60 min,Stand-Up Comedy
9,70304990,Movie,"Good People, September 8, 2017",Henrik Ruben Genz,"James Franco, Kate Hudson, Tom Wilkinson, Omar...","United States, United Kingdom, Denmark, Sweden","September 8, 2017",2014,R,90 min,"Action & Adventure, Thrillers"


In [17]:
# function that will create edges for given movie title and its genres
def addToGraph(movie_name,graph):
    genres=df[df['title']==movie_name]['listed_in'].values[0].rstrip().lower().split(', ')
    for genre in genres:
        graph.add_edge(movie_name.strip(),genre)
    return graph

#function that will create graph for all the movies name
def createGraph():
    graph = nx.Graph()
    for movie_name in df['title']:
        graph=addToGraph(movie_name,graph)
    return graph

In [18]:
graph=createGraph()

In [19]:
print(graph.degree()['Norm of the North: King Sized Adventure, September 9, 2019']) #should be 2 since two genres are assoicated with it
print(graph.degree()['#realityhigh, September 8, 2017']) #shoukd be 1 since 1 genres are assoicated with it

2
1


In [20]:
node2vec = Node2Vec(graph, dimensions=20, walk_length=16, num_walks=10)

Computing transition probabilities:   0%|          | 0/3812 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 10/10 [00:24<00:00,  2.43s/it]


In [22]:
model = node2vec.fit(window=5, min_count=1)

In [23]:
model.wv.get_vector('Ralph Breaks the Internet: Wreck-It Ralph 2, June 11, 2019')

array([-1.0217216 ,  0.48553717, -0.49347752, -0.734115  ,  0.29773647,
       -0.4953884 ,  1.0518061 ,  0.62988544,  0.26297235,  0.19540767,
        0.26046297, -0.31070524,  0.25924087, -0.23208758, -0.12277203,
       -0.0331669 ,  0.29378498, -0.2849092 , -0.19101524, -1.0352458 ],
      dtype=float32)

In [24]:
model.wv.get_vector('Transformer, February 20, 2019')

array([ 0.32111132,  0.02899113, -0.33117306,  0.15517344, -0.4562063 ,
        0.55812687,  1.1921618 ,  0.49982786, -0.26728487,  0.66850716,
        1.3272952 , -0.25055474, -1.042387  , -1.1036918 , -0.28456622,
        0.8215388 ,  1.2093759 ,  0.90895814,  0.13174611, -0.7979148 ],
      dtype=float32)

In [26]:
#generate similiar movies to given genre or title
def print_similiar(name):
    for node, _ in model.wv.most_similar(name):
        print(node)

In [27]:
print_similiar('children & family movies')
#As we can see most movies like "Barbie", "sKrish Trish and Baltiboy" are indeed children movies and shows

You Can Tutu, December 31, 2017
Monster High: Ghouls Rule, March 1, 2018
Balto 3: Wings of Change, April 1, 2018
Barbie: Spy Squad, October 1, 2017
An American Tail: Fievel Goes West, April 1, 2018
Tarzan 2, June 23, 2017
Annabelle Hooper and the Ghosts of Nantucket, August 1, 2017
The Water Horse: Legend of the Deep, November 1, 2018
Welcome to Monster High: The Origin Story, October 1, 2017
Monster High: Freaky Fusion, January 30, 2018


In [28]:
print_similiar('Naruto Shippuden : Blood Prison, September 1, 2017')
# We get results like another naruto series, seven deadly sins which are really good recommendation.

Ghost Pain, February 24, 2019
EVANGELION: DEATH (TRUE)², June 21, 2019
Ghost Tears, February 24, 2019
InuYasha: The Movie 2: The Castle Beyond the Looking Glass, September 1, 2017
Berserk: The Golden Age Arc I - The Egg of the King, October 15, 2016
The End of Evangelion, June 21, 2019
GODZILLA The Planet Eater, January 9, 2019
Inuyasha the Movie - L'isola del fuoco scarlatto, September 1, 2017
Naruto the Movie: Ninja Clash in the Land of Snow, September 1, 2017
InuYasha the Movie: Affections Touching Across Time, August 25, 2018
